## Data Cleaning and Data Prepping for EDA

In [21]:
import sys
from pathlib import Path

root_dir = Path.cwd().parent  # Go one folder up from /notebooks
print("Root directory:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

Root directory: /Users/agalyaayyadurai/Automated-Equity-Valuation


## Data Cleaning 

### Merging Revenue (Original Values) from the original financials_panel.parquet

In [22]:
#Reason: With data merging on the way the rvenue column was trandforme ot log revenue so merging back the original revenue values
import pandas as pd
import numpy as np

df = pd.read_parquet("../data/enriched/financials_annual_th_ohlcv.parquet")
df_orig = pd.read_parquet("../data/gold/financials_panel.parquet")

# ----------------------------------------------------
# Make sure merge keys have the same dtypes in BOTH dfs
# ----------------------------------------------------
for d in (df, df_orig):
    # CIK as plain int64
    d["cik"] = d["cik"].astype("int64")
    # Period as datetime (no timezone)
    d["period"] = pd.to_datetime(d["period"]).dt.tz_localize(None) \
                  if hasattr(d["period"].dtype, "tz") else pd.to_datetime(d["period"])

# ----------------------------------------------------
# Take only the original Revenue, dedup on (cik, period)
# ----------------------------------------------------
rev_orig = (
    df_orig[["cik", "period", "Revenue"]]
    .dropna(subset=["Revenue"])                          # keep only rows with valid Revenue
    .sort_values(["cik", "period"])                      # sort so "last" is the latest if duplicates
    .drop_duplicates(["cik", "period"], keep="last")     # one Revenue per cik-year
    .rename(columns={"Revenue": "Revenue_raw"})
)

print("rev_orig shape:", rev_orig.shape)

# ----------------------------------------------------
# Merge raw Revenue back into enriched df
# ----------------------------------------------------
df = df.merge(
    rev_orig,
    on=["cik", "period"],
    how="left",       # keep all rows from enriched DF
    suffixes=("", "_orig")
)

print("Share of rows with missing Revenue_raw after merge:",
      df["Revenue_raw"].isna().mean())

# ----------------------------------------------------
# Replace transformed Revenue with raw Revenue
# ----------------------------------------------------
df["Revenue"] = df["Revenue_raw"]
df = df.drop(columns=["Revenue_raw"])

# sanity check
print(df["Revenue"].describe())

# ----------------------------------------------------
# Create a clean log Revenue column (for modelling)
# ----------------------------------------------------
# Filter out non-positive Revenue before taking log
df.loc[df["Revenue"] <= 0, "Revenue"] = np.nan
df["Revenue_log"] = np.log(df["Revenue"])

print(df[["Revenue", "Revenue_log"]].head())

# ----------------------------------------------------
# Save cleaned version back to parquet
# ----------------------------------------------------
df.to_parquet("../data/ml/financials_annual_th_ohlcv_clean.parquet", index=False)
print("Saved cleaned DF with raw Revenue restored.")


rev_orig shape: (251596, 3)
Share of rows with missing Revenue_raw after merge: 0.22641658752769864
count    9.775000e+03
mean     3.928612e+09
std      1.861822e+10
min     -1.259785e+11
25%      4.730836e+06
50%      1.015314e+08
75%      9.360970e+08
max      3.943280e+11
Name: Revenue, dtype: float64
        Revenue  Revenue_log
0  3.076471e+10    24.149634
1  3.516672e+10    24.283366
2  3.885126e+10    24.383006
3  3.987391e+10    24.408988
4  2.184800e+10    23.807375
Saved cleaned DF with raw Revenue restored.


### Duplicate CIK & Period

In [23]:
# Check amended filings (duplicate CIK & period)
df[df.duplicated(['cik','period'], keep=False)][['cik','name','period','filed', 'form', 'price_date']].sort_values(['cik','period','filed']).head(20)

,cik,name,period,filed,form,price_date
11521,1740797,AVANT TECHNOLOGIES INC.,2024-03-31,2024-07-03,10-K,2024-07-05 00:00:00-04:00
11522,1740797,AVANT TECHNOLOGIES INC.,2024-03-31,2025-02-19,10-K/A,2024-08-02 00:00:00-04:00


In [24]:
#Removing duplicates keeping the latest ammended form (10-K/A)
#Remove older 10k and keep the latest 10k-a for the abvoe recrod
df = (
    df.sort_values(['cik','period','filed'])
      .drop_duplicates(['cik','period'], keep='last')
)

print(df.shape, "after removing duplicate fiscal years")

(12635, 84) after removing duplicate fiscal years


### Removing CIKs with different company names

In [25]:
# REmoving junk CIKS with different company names
BLACKLIST_CIKS = [
    1098009, 1156784, 1454742, 1462223, 1528188, 1529113, 1584480,
    1604191, 1615999, 1582249, 1593184, 1451512, 1497130, 1498067,
    1511820, 1514056, 1516551, 1520047, 1522767, 1527728, 1528188,
    1534154, 1535469, 1546392, 1551887, 1552189, 1556801, 1557340,
    1557376, 1559053, 1566044, 1568385, 1580262, 1580836, 1618835
]

df = df[~df['cik'].isin(BLACKLIST_CIKS)]

##### 
We removed a small set of firms (35 CIKs) that exhibited clear characteristics of non-operating shell entities, serial reverse mergers, or microcap pivot companies.

These issuers typically change names and business lines in unrelated industries (e.g., from mining to crypto to cannabis), and their financial statements do not represent stable, economically meaningful operations.

Including them would distort model training because their fundamentals contain extreme outliers, zero-revenue periods, and inconsistent reporting practices.

All removals were based on well-documented shell-company histories and did not affect any major listed firms.

| CIK         | Reason (simple)                                                            |
| ----------- | -------------------------------------------------------------------------- |
| **1098009** | Shell pivot: “Crown Marketing” → “America Great Health”                    |
| **1156784** | Reverse-merger shell: CoroWare → CarbonMeta (crypto pivot)                 |
| **1454742** | Shell pivot: HDS International → Good Gaming                               |
| **1462223** | Serial shell mergers: SA Recovery → Truli Media → Recruiter.com            |
| **1528188** | Shell: GOFF CORP → Worldwide NFT (pump-and-dump history)                   |
| **1529113** | Shell churn: Sysorex → Inpixon → XTI Aerospace                             |
| **1584480** | Shell: UpperSolution → Startech Labs (non-operating microcap)              |
| **1604191** | Serial pivots: Azurrx → First Wave Bio → Entero Therapeutics               |
| **1615999** | Shell pivot: TYG Solutions → Kannalife → Neuropathix                       |
| **1582249** | Shell sequence: Active With Me → Rasna → Actavia                           |
| **1593184** | Shell: Findit → BioRegenX (no stable business operations)                  |
| **1451512** | Cannabis shell: Terra Tech → Unrivaled Brands                              |
| **1497130** | Shell: Discovery Gold → GRN Holding                                        |
| **1498067** | Shell churn: ProGaming Platforms → TechCare → Citrine Global               |
| **1511820** | Shell: Globe Net → Stemtech Corp                                           |
| **1514056** | Shell pivot: TraQiQ → Titan Environmental                                  |
| **1516551** | Shell: Nemus Bio → Emerald Bio → Skye Bioscience                           |
| **1520047** | Shell: Pharma Investing News → Immunoclin                                  |
| **1522767** | Shell: Worlds Online → MariMed                                             |
| **1527728** | Shell churn: Dandrit → Enochian → Renovaro → Renovaro Inc                  |
| **1528188** | Shell: GOFF Corp → Worldwide NFT (NFT pump cycle)                          |
| **1534154** | Shell churn: IIM Global → ID Global → Ipsidy → AuthID                      |
| **1535469** | Shell: RealEstate Pathways → NOHO                                          |
| **1546392** | Shell: Media Mechanics → GAWK                                              |
| **1551887** | Shell: Vgrab → Duesenberg Technologies                                     |
| **1552189** | Shell: Exactus → Panacea Life                                              |
| **1556801** | Shell: Oconn Industries → Diamante Minerals                                |
| **1557340** | Shell: Green Meadow → Optec                                                |
| **1557376** | Shell: Organicell → Zeo Scientifix                                         |
| **1559053** | Shell-like: Prothena PLC variant (multiple listings)                       |
| **1566044** | Shell churn: Menlo → VYNE → (frequent restructures, fundamentals unstable) |
| **1568385** | Shell: Bright Mountain → Bright Mountain Media                             |
| **1580262** | Shell: HealthTalk → Right on Brands                                        |
| **1580836** | Shell: Green Supplements → LVYuan                                          |
| **1584480** | Shell: UpperSolution → Startech Labs                                       |
| **1618835** | Shell: Neothetics → Evofem                                                 |


### Shared tickers by two or more companies at the same period

In [26]:
'''
Trying to fix:

The data has ticker-based OHLCV, but company identity is CIK-based.

When a ticker is shared by two CIKs in the same period, there are 3 possibilities:

Both CIKs are genuinely trading under that ticker
→ practically never the case in the examples.

One CIK is the real listed company, the other is a subsidiary / wrong mapping
→ LUMN, CHTR, MRK, MRNA cases.

Two unrelated small caps got wrongly mapped to the same ticker
→ AXON case (Axovant vs Myovant).

Because the prices were pulled by ticker, any “extra” CIK using that ticker in the same period gets wrong prices.

So, must remove the extra CIKs or drop the whole ticker which the code handles
'''

DROP_TICKERS = ['FIL']

DROP_CIKS = [
    1636050, 1679082,    # AXON
    1271833,             # CHTR subsidiary
    794323,              # LUMN subsidiary
    64978,               # MRK wrong entity
    737207               # MRNA wrong entity
]

df = df[~df['ticker'].isin(DROP_TICKERS)]
df = df[~df['cik'].isin(DROP_CIKS)]

##### 
For tickers that mapped to multiple CIKs within the same fiscal period, we examined each case manually. If one CIK corresponded to the true listed equity and the other to a subsidiary or a misclassified issuer, we retained only the valid CIK and removed the invalid one. In cases where the mapping could not be resolved unambiguously (e.g., AXON mapping to two unrelated issuers), we removed all affected records for that ticker.

### Duplicate fiscal years removal

In [27]:
df['fiscal_year'] = df['period'].dt.year

In [28]:
dups = (
    df.groupby(['cik', 'fiscal_year'])
      .size()
      .reset_index(name='n')
      .query('n > 1')
)

dups.head()

,cik,fiscal_year,n
3140,931059,2015,2
8485,1513856,2013,2
9275,1574235,2015,2
10051,1635748,2016,2


In [29]:
# mark bad (cik, fiscal_year) pairs
dups['bad_pair'] = True

df = df.merge(dups[['cik', 'fiscal_year', 'bad_pair']],
              on=['cik', 'fiscal_year'],
              how='left')

# keep only rows that are NOT in bad pairs
df = df[df['bad_pair'].isna()].drop(columns='bad_pair')

In [30]:
df.groupby(['cik', 'fiscal_year']).size().max()
# should be 1

1

### changing price_date dtype

In [31]:
df['price_date'] = df['price_date'].dt.tz_localize(None)

### Filing date > 180d from Fiscal date

In [32]:
df['delay'] = (df['filed'] - df['period']).dt.days
MAX_DELAY = 180
df = df[df['delay'] <= MAX_DELAY].copy()
removed = (df['delay'] > MAX_DELAY).sum()
kept = len(df)
total = len(df)

print(removed, kept, total)
print(df['delay'].describe())
print(df[df['delay'] > 120][['cik','name','period','filed','delay']].head(20))

0 11915 11915
count    11915.000000
mean        72.001511
std         21.458906
min         17.000000
25%         57.000000
50%         70.000000
75%         87.000000
max        180.000000
Name: delay, dtype: float64
         cik                            name     period      filed  delay
269    27093                    DATARAM CORP 2015-04-30 2015-09-11    134
622    73290                   BIOMERICA INC 2021-05-31 2021-10-14    136
707    90721                   PERVASIP CORP 2014-11-30 2015-05-01    152
797   103145           VEECO INSTRUMENTS INC 2018-12-31 2019-05-01    121
902   278165            QUEST SOLUTION, INC. 2017-12-31 2018-05-08    128
903   278165            QUEST SOLUTION, INC. 2018-12-31 2019-06-05    156
1056  352998                  TELIGENT, INC. 2020-12-31 2021-05-04    124
1070  354260                     SOFTECH INC 2014-05-31 2014-10-07    129
1147  709283               QUANTUM CORP /DE/ 2019-03-31 2019-08-06    128
1235  723533  ALLIED VENTURES HOLDINGS COR

In [33]:
(df['delay'] > 150).sum()

89

### Updating OHLCV to Filed+1trading day logic

In [34]:
used_filed_logic = (df['delay'] <= 100).sum()
used_period90_logic = (df['delay'] > 100).sum()

print(used_filed_logic, used_period90_logic)

10814 1101


In [35]:
mask_refresh = df['delay'] > 100
rows_to_refresh = df.loc[mask_refresh]
print(len(rows_to_refresh))
df['price_date'] = pd.to_datetime(df['price_date'], errors='coerce')

1101


In [36]:
from pandas.tseries.offsets import BDay
df['ref_date'] = df['filed'] + BDay(1)

In [37]:
import yfinance as yf
from pandas.tseries.offsets import BDay
import time

# rows needing refresh
mask_refresh = df['delay'] > 100
rows_to_refresh_idx = df.index[mask_refresh]

print("Rows to refresh:", len(rows_to_refresh_idx))

for idx in rows_to_refresh_idx:
    row = df.loc[idx]
    ticker = row['ticker']
    ref_date = row['filed'] + BDay(1)

    # Fetch first trading day OHLCV on/after ref_date
    try:
        hist = yf.Ticker(ticker).history(start=ref_date, end=ref_date + pd.Timedelta(days=10))
        if hist is None or hist.empty:
            continue

        px_date = hist.index[0]
        ohlcv = hist.iloc[0]

        df.loc[idx, 'price_date'] = px_date
        df.loc[idx, 'open'] = ohlcv['Open']
        df.loc[idx, 'high'] = ohlcv['High']
        df.loc[idx, 'low'] = ohlcv['Low']
        df.loc[idx, 'close'] = ohlcv['Close']
        df.loc[idx, 'volume'] = ohlcv['Volume']
    except Exception:
        continue

    # Pause to avoid rate limits
    time.sleep(0.5)

Rows to refresh: 1101


/var/folders/zz/l0v68lmx1_1fxy9jgn7ynjkw0000gn/T/ipykernel_22290/377910942.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2012-04-20 00:00:00-04:00' has dtype incompatible with datetime64[us], please explicitly cast to a compatible dtype first.
  df.loc[idx, 'price_date'] = px_date
$GTBP: possibly delisted; no price data found  (1d 2012-04-17 00:00:00 -> 2012-04-27 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1334635200, endDate = 1335499200")
$TLS: possibly delisted; no price data found  (1d 2020-04-14 00:00:00 -> 2020-04-24 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1586836800, endDate = 1587700800")
$WYGC: possibly delisted; no price data found  (1d 2012-04-23 00:00:00 -> 2012-05-03 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1335153600, endDate = 1336017600")
$VDG: possibly delisted; no price data found  (1d 2012-02-17 00:00:00 -> 2012-02-27 00

In [38]:
df.to_parquet("../data/ml/ticker_ohlcv_updated.parquet")

In [39]:
df.loc[mask_refresh, ['price_date','open','close','volume']].isna().sum()

price_date    90
open          90
close         90
volume        90
dtype: int64

### Removing the null OHLCV, REvenue, OperatingIncome, CommonSharesOutstading records

In [58]:
import pandas as pd
df = pd.read_parquet("../data/ml/ticker_ohlcv_updated.parquet")
df = df.dropna(subset=['price_date', 'open', 'close', 'volume', 'Revenue', 'OperatingIncome', 'CommonSharesOutstanding'])

### Remove TotalAssets and CommonSharesOutstanding == 0

In [59]:
df = df[df["TotalAssets"] > 0]
df = df[df["CommonSharesOutstanding"] > 0]

### Remove negative OHLCV data

In [60]:
df = df[(df['open'] > 0) & (df['close'] > 0) & 
        (df['high'] > 0) & (df['low'] > 0)]

### Remove records with impossible revenue jumps (after manual inspection)

In [61]:
'''Removed because the reported revenue (USD 2 billion) is impossible for an early-stage biotech 
and is almost certainly an XBRL parsing error or reverse-merger contamination - Ayala Pharmaceuticals (CIK 1100397)'''
df = df[~((df['cik'] == 1100397) & (df['period'] == '2023-12-31'))]

In [62]:
import pandas as pd
df = df[~((df['cik'] == 1671858) & (df['period'] == '2024-12-31'))]
'''Removed because the reported revenue (USD 89 million) is inconsistent with ARS Pharmaceuticals’ SEC filings 
and business model. This is almost certainly an XBRL scaling or tagging error.'''

'Removed because the reported revenue (USD 89 million) is inconsistent with ARS Pharmaceuticals’ SEC filings \nand business model. This is almost certainly an XBRL scaling or tagging error.'

In [63]:
# Remove incorrect revenue years for Viewbix/Kidoz (CIK 797542)
df = df[~((df['cik'] == 797542) & (df['period'].isin([
    pd.Timestamp('2022-12-31'),
    pd.Timestamp('2023-12-31'),
    pd.Timestamp('2024-12-31')
])))]
'''Removed Viewbix/Kidoz (CIK 797542) revenue entries for 2022–2024 due to inconsistent XBRL scaling and tagging errors that make the reported values unreliable.'''

'Removed Viewbix/Kidoz (CIK 797542) revenue entries for 2022–2024 due to inconsistent XBRL scaling and tagging errors that make the reported values unreliable.'

In [64]:
df = df[~((df['cik'] == 894871) & (df['period'] == pd.Timestamp('2017-12-31')))]
'''Removed 2017 revenue for TheMaven/Arena Group (CIK 894871) due to a known XBRL tagging inconsistency; 
later-year revenues align correctly with SEC-reported values.'''

'Removed 2017 revenue for TheMaven/Arena Group (CIK 894871) due to a known XBRL tagging inconsistency; \nlater-year revenues align correctly with SEC-reported values.'

In [65]:
df = df[~((df['cik'] == 1501112) & (df['period'] == '2014-08-31'))]
'''2014 revenue of $2,004 is almost certainly reported in the wrong unit'''

'2014 revenue of $2,004 is almost certainly reported in the wrong unit'

In [66]:
df.loc[(df['cik'] == 1070050) & (df['period'] == '2022-12-31'), 'Revenue'] = 450000
'''The 2022 revenue for AppTech Payments (CIK 1070050) was corrected from 450 to 450,000 because the reported figure was missing three zeros.'''

'The 2022 revenue for AppTech Payments (CIK 1070050) was corrected from 450 to 450,000 because the reported figure was missing three zeros.'

### Remove where both TotalLiabilites and ShareHoldersEquity are null

In [67]:
df = df[~(df['TotalLiabilities'].isna() & df['ShareholdersEquity'].isna())]

### Remove impossible NetIncome and OperatingIncome values

In [68]:
df = df[~(df['NetIncome'].abs() > df['TotalAssets'] * 20)]
df = df[~(df['OperatingIncome'].abs() > df['TotalAssets'] * 20)]

In [69]:
df.to_parquet("../data/ml/financials_annual_th_ohlcv_clean.parquet", index=False)

## Archive

In [15]:
keep_cols = [
    # identifiers
    'cik', 'ticker', 'name', 'period', 'filed', 'price_date',

    # market data
    'open', 'high', 'low', 'close', 'volume', 'CommonSharesOutstanding',

    # core fundamentals
    'Revenue', 'GrossProfit', 'OperatingIncome', 'NetIncome',
    'OperatingExpenses', 'CostOfRevenue',
    'TotalAssets', 'TotalLiabilities', 'ShareholdersEquity',
    'TotalCurrentAssets', 'TotalCurrentLiabilities',
    'CashAndCashEquivalents', 'AccountsReceivable', 'Inventory',
    'PPandE', 'LongTermDebt', 'ShortTermDebt',
    'OtherCurrentLiabilities', 'OtherNoncurrentLiabilities',

    # cash flow items
    'CFO', 'CFI', 'CFF', 'CapEx', 'DepAmortCF', 'StockBasedComp',
    'DebtIssuance', 'DebtRepayment', 'ShareRepurchase',
    'EquityIssuance', 'IncomeTaxesPaid', 'InterestPaid',

    # R&D and SG&A
    'RND', 'SGA'
]

df = df[keep_cols].copy()


In [2]:
import pandas as pd

# Loading the financial statements of all the companies
file_path = Path("../data/gold/financials_ticker_panel.parquet")
df_raw = pd.read_parquet(file_path)

In [3]:
from src.data_prep.prep_config import PREP_CONFIG
from src.data_prep.data_prep import process_fundamentals

# Run the full cleaning pipeline
df_clean, scalers = process_fundamentals(df_raw, config=PREP_CONFIG, verbose=True)

df_clean.to_parquet("../data/financials_clean.parquet", compression="gzip", index=False)

import pickle
pickle.dump(scalers, open("prep_scalers.pkl", "wb"))

# 3. Save scalers to reuse for future quarters (optional)
# e.g. with pickle or json

[WARN] Missing expected columns: ['instance']
[INFO] Extra columns present (will be ignored by pipeline): ['ADSOutstanding', 'APIC', 'AccumulatedOCI', 'AcquireIntangibles', 'AntidilutiveExcluded', 'BusinessAcquisitions', 'CommonSharesAuthorized', 'CommonSharesIssued', 'CommonStockParValue', 'CommonStockValue', 'ComprehensiveIncome', 'DebtExtinguishmentGainLoss', 'DepAmort', 'EquityMethodIncome', 'FXEffect', 'Impairment', 'LeaseLiabilityCurrent', 'LeaseLiabilityNoncurrent', 'LeaseROUAsset', 'NoncontrollingInterest', 'OtherIncomeExpense', 'OtherNoncurrentLiabilities', 'PreferredSharesAuthorized', 'PreferredSharesDesignated', 'PreferredSharesIssued', 'PreferredSharesOutstanding', 'PreferredStockParValue', 'PreferredStockValue', 'PretaxIncome', 'ROUAssetNoncash', 'SharesDesignated', 'SharesIssuable', 'SharesReservedForFutureIssuance', 'ShortTermInvestments', 'TemporaryEquity', 'TemporaryEquityShares', 'TotalNoncurrentAssets', 'TreasuryShares', 'TreasuryStock', 'UnitsIssued', 'UnitsOutstand

In [5]:
# Merging hte ticker for each CIK

from src.data_prep.cik_ticker import load_sec_cik_ticker_exchange, infer_cik_tickers_from_fsds_zips
from src.data_extract.bronze_extractor.fsds_loader import load_fsds_from_zip   # adjust this import

# 1. Load SEC mapping 
sec_map = load_sec_cik_ticker_exchange("../data/company_tickers_exchange.json")

# 2. Infer tickers from instance prefixes in all FSDS zips
zips_root = "../data/raw"  

cik_inf = infer_cik_tickers_from_fsds_zips(
    zips_root=zips_root,
    load_fsds_from_zip=load_fsds_from_zip,
)

print(cik_inf.head())
print("[INFO] Inferred CIKs:", len(cik_inf))


[INFO] Processing zip: ../data/raw/2025q2.zip
[INFO] Processing zip: ../data/raw/2023q4.zip
[INFO] Processing zip: ../data/raw/2018q2.zip
[INFO] Processing zip: ../data/raw/2018q3.zip
[INFO] Processing zip: ../data/raw/2025q1.zip
[INFO] Processing zip: ../data/raw/2018q1.zip
[INFO] Processing zip: ../data/raw/2021q4.zip
[INFO] Processing zip: ../data/raw/2023q2.zip
[INFO] Processing zip: ../data/raw/2018q4.zip
[INFO] Processing zip: ../data/raw/2021q1.zip
[INFO] Processing zip: ../data/raw/2023q3.zip
[INFO] Processing zip: ../data/raw/2023q1.zip
[INFO] Processing zip: ../data/raw/2021q3.zip
[INFO] Processing zip: ../data/raw/2021q2.zip
[INFO] Processing zip: ../data/raw/2019q1.zip
[INFO] Processing zip: ../data/raw/2024q1.zip
[INFO] Processing zip: ../data/raw/2020q4.zip
[INFO] Processing zip: ../data/raw/2022q4.zip
[INFO] Processing zip: ../data/raw/2019q2.zip
[INFO] Processing zip: ../data/raw/2024q2.zip
[INFO] Processing zip: ../data/raw/2024q3.zip
[INFO] Processing zip: ../data/raw

In [6]:
# 1. Merge SEC mapping
df_clean["cik"] = pd.to_numeric(df_clean["cik"], errors="coerce").astype("Int64")
sec_map["cik"] = pd.to_numeric(sec_map["cik"], errors="coerce").astype("Int64")

df_with_ticker = df_clean.merge(
    sec_map[["cik", "ticker_sec"]],
    on="cik",
    how="left"
)
df_with_ticker = df_with_ticker.rename(columns={"ticker_sec": "ticker"})

# 2. Merge instance-inferred mapping
cik_inf["cik"] = pd.to_numeric(cik_inf["cik"], errors="coerce").astype("Int64")

df_with_ticker = df_with_ticker.merge(
    cik_inf[["cik", "ticker_inferred"]],
    on="cik",
    how="left"
)

# 3. For rows where SEC ticker is missing, fill from inferred
df_with_ticker["ticker"] = df_with_ticker["ticker"].fillna(df_with_ticker["ticker_inferred"])
df_with_ticker = df_with_ticker.drop(columns=["ticker_inferred"])

# 4. Check coverage
coverage = df_with_ticker["ticker"].notna().mean() * 100
print(f"[INFO] Combined SEC + instance-inferred match rate: {coverage:.2f}%")

df_with_ticker[["cik", "name", "fy", "period", "ticker"]].head()

[INFO] Combined SEC + instance-inferred match rate: 97.94%


,cik,name,fy,period,ticker
0,38074,FOREST LABORATORIES INC,2008,2009-03-31,FRX
1,1002638,OPEN TEXT CORP,2009,2009-06-30,OTEX
2,833444,TYCO INTERNATIONAL LTD /BER/,2009,2009-09-30,JCI
3,1385157,TYCO ELECTRONICS LTD.,2009,2009-09-30,TEL
4,1024478,ROCKWELL AUTOMATION INC,2009,2009-09-30,ROK


In [7]:
missing_count = df_with_ticker["ticker"].isna() | (df_with_ticker["ticker"] == "None")
missing_count.sum()

1688

## Prep the data with ticker

In [9]:
df_with_ticker.to_parquet("../data/gold/financials_ticker_panel.parquet")

In [28]:
import pandas as pd
df_with_ticker = pd.read_parquet("../data/gold/financials_ticker_panel.parquet")

In [37]:
# Checking the period column of one ticker
filt = df_with_ticker[df_with_ticker["ticker"] == "META"]
filt.period

8709    2012-12-31
14961   2013-12-31
21161   2014-12-31
27340   2015-12-31
33087   2016-12-31
38671   2017-12-31
44112   2018-12-31
49441   2019-12-31
54625   2020-12-31
59971   2021-12-31
66117   2022-12-31
72059   2023-12-31
77618   2024-12-31
Name: period, dtype: datetime64[ns]

## Prep the Tech + Healthcare data and attach OHLCV for ML model

In [1]:
import sys
from pathlib import Path
import pandas as pd

root_dir = Path.cwd().parent  # Go one folder up from /notebooks
print("Root directory:", root_dir)

if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))
    
df_th = pd.read_parquet("../data/enriched/financials_annual_th_ticker.parquet")

Root directory: /Users/agalyaayyadurai/Automated-Equity-Valuation


In [ ]:
#Remove all rows that have ticker as "DELISTED"
df_th = df_th[~(df_th["ticker"] == "DELISTED")]
df_th

In [3]:
# Filed and period proper datetime
import pandas as pd
import yfinance as yf
from pandas.tseries.offsets import BDay  # business day offset

# Clean date columns once
df_th["filed_dt"] = pd.to_datetime(df_th["filed"], errors="coerce")
df_th["period_dt"] = pd.to_datetime(df_th["period"], errors="coerce")

In [3]:
import pandas as pd
import numpy as np
import yfinance as yf
from pandas.tseries.offsets import BDay
import os
import time

def safe_first_ohlcv_after(ticker, ref_date, max_days=10):
    """Return OHLCV for first trading day on/after ref_date or (None, None)."""
    if pd.isna(ref_date):
        return None, None

    start = ref_date
    end = ref_date + pd.Timedelta(days=max_days)

    try:
        hist = yf.Ticker(ticker).history(start=start, end=end)
        if hist is None or hist.empty:
            return None, None
        first_day = hist.index.min()
        return first_day, hist.loc[first_day]
    except Exception:
        return None, None


def attach_ohlcv_restart_safe(
    input_parquet,
    output_parquet,
    ticker_col="ticker",
    filed_candidates=("filed_dt", "filed"),
    period_candidates=("period_dt", "period"),
    max_filing_delay_days=100,
    fallback_days_after_period=90,
):
    """
    Crash-safe OHLCV attachment:
    - Detects filed/period column names automatically from candidates.
    - Uses filed/period rule:
        if |filed - period| > max_filing_delay_days -> ref = period + fallback_days_after_period * BDay
        else -> ref = filed + 1 * BDay
    - For each row, gets first trading day's OHLCV on/after ref date.
    - Saves progress after EVERY row to output_parquet.
    - If output_parquet exists, it loads that and continues from where it left off.
    """

    # 1) Load base df from input
    df_input = pd.read_parquet(input_parquet).copy()

    # 2) Detect filed/period column names
    filed_col = next((c for c in filed_candidates if c in df_input.columns), None)
    period_col = next((c for c in period_candidates if c in df_input.columns), None)

    if filed_col is None or period_col is None:
        raise KeyError(
            f"Could not find filed/period columns. "
            f"Tried filed candidates={filed_candidates}, period candidates={period_candidates}. "
            f"Available columns: {list(df_input.columns)}"
        )

    print(f"Using filed column: {filed_col}")
    print(f"Using period column: {period_col}")

    # 3) If output parquet exists, load it to resume progress; else start from input
    if os.path.exists(output_parquet):
        print("Restart detected → loading progress from output parquet…")
        df = pd.read_parquet(output_parquet).copy()

        # Ensure all columns from input exist in df
        for col in df_input.columns:
            if col not in df.columns:
                df[col] = df_input[col]
    else:
        print("Starting fresh → no existing progress file.")
        df = df_input.copy()

    # 4) Basic cleaning
    df = df[df[ticker_col].notna() & (df[ticker_col] != "DELISTED")].copy()
    df[ticker_col] = df[ticker_col].astype(str).str.strip()

    df[filed_col] = pd.to_datetime(df[filed_col], errors="coerce")
    df[period_col] = pd.to_datetime(df[period_col], errors="coerce")

    # 5) Ensure OHLCV columns exist
    for col in ["price_date", "open", "high", "low", "close", "volume"]:
        if col not in df.columns:
            df[col] = np.nan

    # 6) Loop through rows, skip those already processed
    total = len(df)
    print(f"Total rows after cleaning: {total}")

    for i, idx in enumerate(df.index):
        row = df.loc[idx]

        # Skip if we already have price_date (this is what makes it restart-safe)
        if not pd.isna(row["price_date"]):
            continue

        ticker = row[ticker_col]
        filed_dt = row[filed_col]
        period_dt = row[period_col]

        if pd.isna(filed_dt) or pd.isna(period_dt):
            # Can't compute difference → skip
            continue

        diff_days = abs((filed_dt - period_dt).days)

        if diff_days > max_filing_delay_days:
            ref_date = period_dt + BDay(fallback_days_after_period)
        else:
            ref_date = filed_dt + BDay(1)

        px_date, ohlcv = safe_first_ohlcv_after(ticker, ref_date, max_days=10)

        if ohlcv is not None:
            df.loc[idx, "price_date"] = pd.to_datetime(px_date)
            # Fill OHLCV if the columns are present
            for src, dst in [
                ("Open", "open"),
                ("High", "high"),
                ("Low", "low"),
                ("Close", "close"),
                ("Volume", "volume"),
            ]:
                if src in ohlcv.index:
                    df.loc[idx, dst] = ohlcv[src]

        # Save progress after each row
        df.to_parquet(output_parquet, index=False)

        # Gentle sleep to avoid rate limiting
        time.sleep(0.25)

        if i % 50 == 0:
            print(f"{i}/{total} rows processed...")

    print("Done! Saved:", output_parquet)
    return df


In [4]:
df_clean = attach_ohlcv_restart_safe(
    "../data/enriched/financials_annual_th_ticker.parquet",
    "../data/enriched/financials_annual_th_ohlcv.parquet"
)

Using filed column: filed
Using period column: period
Restart detected → loading progress from output parquet…
Total rows after cleaning: 12636


$FYBR: possibly delisted; no price data found  (1d 2013-02-28 00:00:00 -> 2013-03-10 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1362027600, endDate = 1362891600")
$FYBR: possibly delisted; no price data found  (1d 2014-02-28 00:00:00 -> 2014-03-10 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1393563600, endDate = 1394424000")
$FYBR: possibly delisted; no price data found  (1d 2015-02-26 00:00:00 -> 2015-03-08 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1424926800, endDate = 1425790800")
$FYBR: possibly delisted; no price data found  (1d 2016-02-26 00:00:00 -> 2016-03-07 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1456462800, endDate = 1457326800")
$FYBR: possibly delisted; no price data found  (1d 2017-03-02 00:00:00 -> 2017-03-12 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1488430800, endDate = 1489294800")
$FYBR: possibly delisted; no price data found  (1d 2018-03-02 00:00:00 -> 2018-03-12 00:00:00) 

550/12636 rows processed...


$CTDD: possibly delisted; no price data found  (1d 2015-03-02 00:00:00 -> 2015-03-12 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1425272400, endDate = 1426132800")
$CTDD: possibly delisted; no price data found  (1d 2016-03-02 00:00:00 -> 2016-03-12 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1456894800, endDate = 1457758800")
$CTDD: possibly delisted; no price data found  (1d 2017-03-03 00:00:00 -> 2017-03-13 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1488517200, endDate = 1489377600")
$NTY: possibly delisted; no price data found  (1d 2012-11-28 00:00:00 -> 2012-12-08 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1354078800, endDate = 1354942800")
$NTY: possibly delisted; no price data found  (1d 2013-11-27 00:00:00 -> 2013-12-07 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1385528400, endDate = 1386392400")
$NTY: possibly delisted; no price data found  (1d 2014-11-25 00:00:00 -> 2014-12-05 00:00:00)
$NT

1050/12636 rows processed...


...

12550/12636 rows processed...


$KHOB: possibly delisted; no price data found  (1d 2024-01-31 00:00:00 -> 2024-02-10 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1706677200, endDate = 1707541200")
$GLTK: possibly delisted; no price data found  (1d 2023-05-05 00:00:00 -> 2023-05-15 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1683259200, endDate = 1684123200")
$MGNC: possibly delisted; no price data found  (1d 2024-09-03 00:00:00 -> 2024-09-13 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1725336000, endDate = 1726200000")


12600/12636 rows processed...


$NXNT: possibly delisted; no price data found  (1d 2024-10-02 00:00:00 -> 2024-10-12 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1727841600, endDate = 1728705600")
$ALXY: possibly delisted; no price data found  (1d 2024-07-04 00:00:00 -> 2024-07-14 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1720065600, endDate = 1720929600")
$ALXY: possibly delisted; no price data found  (1d 2025-05-30 00:00:00 -> 2025-06-09 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1748577600, endDate = 1749441600")
$IMAA: possibly delisted; no price data found  (1d 2024-07-30 00:00:00 -> 2024-08-09 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1722312000, endDate = 1723176000")
$LDXC: possibly delisted; no price data found  (1d 2024-08-20 00:00:00 -> 2024-08-30 00:00:00) (Yahoo error = "Data doesn't exist for startDate = 1724126400, endDate = 1724990400")
$GAFC: possibly delisted; no price data found  (1d 2024-11-11 00:00:00 -> 2024-11-21 00:00:00)


Done! Saved: ../data/enriched/financials_annual_th_ohlcv.parquet


In [6]:
df_ohlcv = pd.read_parquet("../data/enriched/financials_annual_th_ohlcv.parquet")